## Урок 5. Проверка статистических гипотез. P-значения. Доверительные интервалы. A/B-тестирование

In [127]:
import pandas as pd
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt

### Задача 1

#### Известно, что генеральная совокупность распределена нормально с известным средним квадратическим отклонением 16. Найти доверительный интервал для оценки математического ожидания mu с надёжностью 0.95 (доверительная информация), если выборочное среднее равно 80, а объём равен 256.

In [22]:
p = 0.95
alpha = 1 - p
means = 80
n = 256
sg = 16

In [23]:
t1 = st.norm.ppf(alpha / 2)
t2 = st.norm.ppf(1 - alpha / 2)

t1, t2

(-1.959963984540054, 1.959963984540054)

Строим доверительный интервал с известной дисперсией (формула стр 18 PDF презентации урок5_презентация.pdf)

In [26]:
P_left = means + t1 * sg / np.sqrt(n)
P_right = means + t2 * sg / np.sqrt(n)
print (f'Доверительный интервал: {P_left} - {P_right}')

Доверительный интервал: 78.04003601545995 - 81.95996398454005


### Задача 2

#### Продавец утверждает, что средний вес пачки печенья составляет 200 г. Из партии извлечена выборка из 10 пачек. Вес каждой пачки составляет: 202, 203, 199, 197, 195, 201, 200, 204, 194, 190 Известно, что их веса распределены нормально.

#### 1. Верно ли утверждение продавца, если учитывать, что уровень значимости равен 1%?

#### 2. Найдите P-значение для данного теста.

In [80]:
n = 10
mu = 200
samples = np.array([202, 203, 199, 197, 195, 201, 200, 204, 194, 190])
alpha = 0.01
means = samples.mean()
sg = samples.std(ddof=1)

In [81]:
t1 = st.t.ppf(alpha / 2, df=n - 1)
t2 = st.t.ppf(1 - alpha / 2, df=n - 1)

t1, t2

(-3.24983554401537, 3.2498355440153697)

In [82]:
def statistic(samples):
    return (means - mu) / (sg / np.sqrt(n))

In [83]:
t = statistic(samples)
t

-1.0651074037450896

###### t не входит в критическую область -> утверждение продавца верно

In [84]:
p_left = st.t.cdf(t, df=n - 1)
p_right = 1 - st.t.cdf(t, df=n - 1)

pvalue = 2 * min(p_left, p_right)

print (f'P-значение: {pvalue}')

P-значение: 0.31456660352472854


На практике использовать P-значения можно так: если выбранный нами уровень значимости меньше P-значения, полученного в результате теста, то гипотезу можно принять.

##### pvalue > alpha -> гипотезу можно принять

### Задача 3

#### Утверждается, что шарики для подшипников, изготовленные автоматическим станком, имеют средний диаметр 17 мм. Используя уровень значимости 0.025 и правостороннюю критическую область, проверить данную гипотезу при условии, что в выборке из 100 шариков средний диаметр оказался равным 17.5 мм, а дисперсия известна и равна 4 мм.

In [85]:
mu = 17
alpha = 0.025
n = 100
sg = np.sqrt(4) #сигма это корень из дисперсии
means = 17.5

In [86]:
t_right = st.norm.ppf(1 - alpha) #считаем одну строну так как задана только правосторонняя критическая область
t_right

1.959963984540054

In [87]:
Z = (means - mu) / (sg / np.sqrt(n))
Z

2.5

In [88]:
p_right = 1 - st.norm.cdf(Z)
p_right

0.006209665325776159

##### p_right < alpha -> гипотеза mu = 17 неверна

### Задача 4 (Дополнительно)

#### Дана выборка диаметров подшипников из примера 1 с занятия 5:

#### samples = [0.6603, 0.9466, 0.5968, 1.3792, 1.5481, 0.7515, 1.0681, 1.1134, 1.2088, 1.701 , 1.0282, 1.3579, 1.0191, 1.1784, 1.1168, 1.1372, 0.7273, 1.3958, 0.8665, 1.5112, 1.161 , 1.0232, 1.0865, 1.02 ]

#### Предполагая, что диаметры подшипников распределены нормально, проверьте гипотезу о том, что дисперсия случайной величины равна 0.0625 при уровне значимости alpha = 0.05. Что для этого нужно знать:

#### 1. Альтернативная гипотеза двухсторонняя.

#### 2. Статистика для теста: H = (n - 1) * sample_variance / variance, где n - число элементов в выборке, sample_variance - несмещённая оценка дисперсии, variance - утверждаемая нулевой гипотезой дисперсия.

#### 3. Эта статистика в предположении верности нулевой гипотезы имеет распределение хи-квадрат с параметром df = n - 1. Её квантили можно найти с помощью функции scipy.stats.chi2.ppf

In [153]:
samples = np.array([0.6603, 0.9466, 0.5968, 1.3792, 1.5481, 0.7515, 1.0681, 1.1134, 1.2088, 1.701 , 1.0282, 1.3579, 
           1.0191, 1.1784, 1.1168, 1.1372, 0.7273, 1.3958, 0.8665, 1.5112, 1.161 , 1.0232, 1.0865, 1.02])
alpha = 0.05
variance = 0.0625
n = samples.shape[0]
sample_variance = samples.var(ddof = 1)  # ddof = 1 - несмещенная оценка дисперсии

Нулевая гипотеза: распределение c дисперсией случайной величины равной 0.0625

In [142]:
t1 = st.chi2.ppf(alpha / 2, df = n - 1)
t2 = st.chi2.ppf(1 - alpha / 2, df = n - 1)

t1, t2

(11.688551922452438, 38.0756272503558)

In [143]:
H = (n - 1) * sample_variance / variance
H

28.720541753333343

#### H не принадлежит критической области -> гипотеза принимается

Проверяем результат через P-значения

In [146]:
p_left = st.chi2.cdf(H, df = n - 1)
p_right = 1 - st.chi2.cdf(H, df = n - 1)

pvalue = 2 * min(p_left, p_right)

print (pvalue, alpha)

0.37950875392024197 0.05


#### pvalue > alpha -> подтверждаем принятие нулевой гипотезы